In [1]:
import numpy as np
import pandas as pd
from config import paths_to_cleaned_data
from datetime import timedelta, datetime
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

In [2]:
def extract(paths: dict) -> tuple:

    """Extract data from a given paths"""

    item_cat = pd.read_pickle(paths['item_cat'])
    items = pd.read_pickle(paths['items'])
    sales_train = pd.read_pickle(paths["sales_train"])
    shops = pd.read_pickle(paths["shops"])
    test = pd.read_pickle(paths["test"])
    return sales_train, item_cat, shops, items, test

In [3]:
%%time
data, item_cat, shops, items, test = extract(paths_to_cleaned_data)
supplementary_tables = (items, item_cat, shops)

CPU times: user 24.1 ms, sys: 73.3 ms, total: 97.4 ms
Wall time: 95 ms


In [4]:
class Validator:
    
    def __init__(self, schema, statistics: pd.DataFrame):
        self.schema = list(schema)
        self.stats = statistics
    
    def _check_schema(self, df):
        assert list(df.columns) == self.schema,\
        TypeError(f"DataFrame schema doesn't correspond required format.\n Required schema: {self.schema}")
        
    def _check_nulls(self, df):
        assert ~df.isnull().values.any(), ValueError("Data contains missing values")
        
    def _check_outliers(self, df, num_std=3):
        columns = df.dtypes[(df.dtypes == 'int64')|(df.dtypes == 'float64')].index
        for col in columns:     
            if (df[f'{col}'] - self.stats[f'{col}']['mean']).abs().max() > num_std*self.stats[f'{col}']['std']:
                raise ValueError(f"{col} contains outliers")
                
    def _check_date_type(self, df):
        assert df.date.dtype == 'datetime64[ns]', TypeError("Index should be of type Timestamp")

    def _check_date_range(self, df):
        if (df.date.min() < pd.Timestamp('2013-01-01')) or (df.date.max() > pd.Timestamp('2022-12-31')):
            raise ValueError("Data should be within the range of 2013-01-01 and 2022-12-31")

        
    def validate(self, df):
        self._check_schema(df)
        self._check_nulls(df)
        self._check_outliers(df, num_std=200)
        self._check_date_type(df)
        self._check_date_range(df)
        return True

In [5]:
val = Validator(schema = data.columns, statistics = data.describe())
success = val.validate(data)
print("Validation succesfully passed!") if success else print(":(")

Validation succesfully passed!


In [6]:
st = sales_train.groupby(['date','shop_id', 'item_id'], as_index=False)\
                 .sum(numeric_only=True)
# st.set_index(['shop_id', 'item_id'], inplace=True)

In [56]:
st.shift(periods=1)

date_block_num  item_price  item_cnt_day
date       shop_id item_id                                          
2013-01-01 2       991                 NaN         NaN           NaN
                   1472                0.0        99.0           1.0
                   1905                0.0      2599.0           1.0
                   2920                0.0       249.0           1.0
                   3320                0.0       599.0           2.0
...                                    ...         ...           ...
2015-10-31 59      20730              33.0       299.0           3.0
                   20866              33.0       499.0           1.0
                   20949              33.0       699.0           1.0
                   21369              33.0         5.0           1.0
                   21449              33.0       169.0           1.0

[2935785 rows x 3 columns]

In [7]:
st

,date,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,2013-01-01,2,991,0,99.0,1.0
1,2013-01-01,2,1472,0,2599.0,1.0
2,2013-01-01,2,1905,0,249.0,1.0
3,2013-01-01,2,2920,0,599.0,2.0
4,2013-01-01,2,3320,0,1999.0,1.0
...,...,...,...,...,...,...
2935780,2015-10-31,59,20730,33,499.0,1.0
2935781,2015-10-31,59,20866,33,699.0,1.0
2935782,2015-10-31,59,20949,33,5.0,1.0
2935783,2015-10-31,59,21369,33,169.0,1.0


In [8]:
st.assign(date=st.date+timedelta(days=7))

,date,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,2013-01-08,2,991,0,99.0,1.0
1,2013-01-08,2,1472,0,2599.0,1.0
2,2013-01-08,2,1905,0,249.0,1.0
3,2013-01-08,2,2920,0,599.0,2.0
4,2013-01-08,2,3320,0,1999.0,1.0
...,...,...,...,...,...,...
2935780,2015-11-07,59,20730,33,499.0,1.0
2935781,2015-11-07,59,20866,33,699.0,1.0
2935782,2015-11-07,59,20949,33,5.0,1.0
2935783,2015-11-07,59,21369,33,169.0,1.0


In [20]:
lag7 = st.merge(st.assign(date=st.date+timedelta(days=7)),
         on = ['date','shop_id', 'item_id','date_block_num', 'item_price'],
         how = 'left',
         suffixes = ['', '_7_days_ago']).fillna(0)
lag7.head(100)

,date,shop_id,item_id,date_block_num,item_price,item_cnt_day,item_cnt_day_7_days_ago
0,2013-01-01,2,991,0,99.0,1.0,0.0
1,2013-01-01,2,1472,0,2599.0,1.0,0.0
2,2013-01-01,2,1905,0,249.0,1.0,0.0
3,2013-01-01,2,2920,0,599.0,2.0,0.0
4,2013-01-01,2,3320,0,1999.0,1.0,0.0
5,2013-01-01,2,4464,0,599.0,1.0,0.0
6,2013-01-01,2,4724,0,1399.0,1.0,0.0
7,2013-01-01,2,5649,0,2190.0,1.0,0.0
8,2013-01-01,2,6911,0,599.0,1.0,0.0
9,2013-01-01,2,6916,0,999.5,1.0,0.0


In [21]:
lag7_gr = lag7.groupby(['date_block_num','shop_id', 'item_id'], as_index=False)\
                 .sum(numeric_only=True)
lag7_gr.head(100)

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_cnt_day_7_days_ago
0,0,2,27,2499.000000,1.0,0.0
1,0,2,33,499.000000,1.0,0.0
2,0,2,317,299.000000,1.0,0.0
3,0,2,438,299.000000,1.0,0.0
4,0,2,471,798.000000,2.0,0.0
5,0,2,481,330.000000,1.0,0.0
6,0,2,482,3300.000000,1.0,0.0
7,0,2,484,600.000000,2.0,0.0
8,0,2,491,600.000000,1.0,0.0
9,0,2,534,798.000000,2.0,0.0


In [17]:
lag7.iloc[100000:].head(100)

,date,shop_id,item_id,date_block_num,item_price,item_cnt_day,item_cnt_day_7_days_ago
100000,2013-01-26,54,15719,0,149.00,1.0,NaN
100001,2013-01-26,54,15729,0,399.00,3.0,1.0
100002,2013-01-26,54,15730,0,1499.00,1.0,NaN
100003,2013-01-26,54,15769,0,799.00,1.0,NaN
100004,2013-01-26,54,16071,0,249.00,1.0,NaN
100005,2013-01-26,54,16135,0,249.00,1.0,NaN
100006,2013-01-26,54,16184,0,249.00,1.0,NaN
100007,2013-01-26,54,16227,0,379.00,1.0,NaN
100008,2013-01-26,54,16439,0,299.00,1.0,NaN
100009,2013-01-26,54,16450,0,399.00,3.0,8.0


In [6]:
class FeatureExtractor:
    
    def __init__(self, data):
        self.data = data
    
    
    def _add_lags(self, df, period):
        merge_cols = list(df.columns)
        merge_cols.remove('item_cnt_day')
        lagged_df = df.merge(df.assign(date_block_num = df.date_block_num + period),
                         on = merge_cols,
                         how = 'left',
                         suffixes = ['', f'_{period}_month_ago'])\
                      .fillna(0)
        
        return lagged_df
    
    
    def extract(self, supplementary_tables):
        
        items, item_cat, shops = supplementary_tables
        
        # adding week lag
        st = self.data.groupby(['date','shop_id', 'item_id','date_block_num'], as_index=False).sum(numeric_only=True)
        df = st.merge(st.assign(date=st.date+timedelta(days=7)),
                         on = ['date','shop_id', 'item_id','date_block_num', 'item_price'],
                         how = 'left',
                         suffixes = ['', '_7_days_ago'])\
               .fillna(0)\
               .groupby(['date_block_num','shop_id', 'item_id'], as_index=False)\
               .sum(numeric_only=True)
        
        # adding 1 month and 1 year lags
        df = self._add_lags(df, period=1)
        df = self._add_lags(df, period=12)
        
        # reduced item_cat
        red_item_cat = item_cat.copy() 
        red_item_cat.loc[0, 'item_category_name'] = 'PC-Гарнитуры Наушники'
        red_item_cat["item_category_name"] = red_item_cat['item_category_name'].str.replace("-|\(.*\)|[A-Za-z0-9]+", '', regex=True)
        red_item_cat["item_category_name"] = red_item_cat['item_category_name'].str.replace(" .*", '', regex=True)
        red_item_cat.rename(columns = {"item_category_name":"short_item_category_name"}, inplace=True)

        # reduced shops
        red_shops = shops.copy() 
        red_shops["shop_name"] = red_shops['shop_name'].str.replace(" .*", '', regex=True)
        red_shops.rename(columns={"shop_name":"shop_city"}, inplace=True)

        # add category and city's
        df = df.merge(items, on='item_id', how='left')\
                 .merge(red_item_cat, on='item_category_id', how='left')\
                 .merge(red_shops, on='shop_id', how='left')
        df = df.drop(columns = ['item_category_id', 'item_name'])
        
        # feature engineering
        df['is_moscow'] = (df.shop_city=='Москва').astype(int)
        df = df.merge(df.groupby(["date_block_num", "is_moscow"],  as_index=False)\
                                         .item_cnt_day\
                                         .mean(),
                                 on = ['date_block_num', 'is_moscow'],
                                 how = 'left',
                                 suffixes = ['', '_']
                                ).rename({"item_cnt_day_":"city_revenue_this_month"}, axis=1)
        df = pd.get_dummies(df, columns=['short_item_category_name'], prefix='category')
        df = df.drop(columns=['shop_city'])
        
        # renaming target
        df.rename({"item_cnt_day":"y"}, axis=1, inplace=True)
        
        return df

In [156]:
X = data.loc[(data.date >= "2013-01-01") & (data.date < "2013-05-01")]
fex = FeatureExtractor(X)
dt = fex.extract(supplementary_tables)
split_block = dt.date_block_num.max()
dt.loc[dt.date_block_num >= split_block]

3
3
3
3
3
3


,date_block_num,shop_id,item_id,item_price,y,item_cnt_day_7_days_ago,item_cnt_day_1_month_ago,item_cnt_day_12_month_ago,is_moscow,city_revenue_this_month,category_Аксессуары,category_Гарнитуры,category_Доставка,category_Игровые,category_Игры,category_Карты,category_Кино,category_Книги,category_Музыка,category_Подарки,category_Программы,category_Служебные,category_Чистые,category_Элементы
187135,3,2,31,699.0,1.0,0.0,0.0,0.0,0,1.830673,0,0,0,0,0,0,1,0,0,0,0,0,0,0
187136,3,2,464,299.0,1.0,0.0,0.0,0.0,0,1.830673,0,0,0,0,0,0,0,1,0,0,0,0,0,0
187137,3,2,482,3300.0,1.0,0.0,1.0,0.0,0,1.830673,0,0,0,0,0,0,0,0,0,0,1,0,0,0
187138,3,2,484,900.0,3.0,1.0,0.0,0.0,0,1.830673,0,0,0,0,0,0,0,0,0,0,1,0,0,0
187139,3,2,485,600.0,2.0,0.0,0.0,0.0,0,1.830673,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241768,3,59,22016,299.0,1.0,0.0,1.0,0.0,0,1.830673,0,0,0,0,0,0,1,0,0,0,0,0,0,0
241769,3,59,22020,798.0,2.0,0.0,0.0,0.0,0,1.830673,0,0,0,0,0,0,1,0,0,0,0,0,0,0
241770,3,59,22076,798.0,8.0,0.0,0.0,0.0,0,1.830673,0,0,0,0,1,0,0,0,0,0,0,0,0,0
241771,3,59,22088,237.0,3.0,0.0,5.0,0.0,0,1.830673,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [141]:
X.date_block_num.max()

3

In [7]:
class SaleCV:
    
    def __init__(self, split_dates, metrics):
        self.split_dates = split_dates
        self.metrics = metrics
        
        cols = ['Model_name', 'train_window', 'test_window']
        train_metrics_cols = ["train_"+str(x).split()[1] for x in metrics]
        test_metrics_cols = ["test_"+str(x).split()[1] for x in metrics]
        cols.extend(train_metrics_cols)
        cols.extend(test_metrics_cols)
        
        self.history = pd.DataFrame(columns=cols)
        
    def _get_batch(self, data, left_boundary='2013-01-01', right_boundary='2015-11-01'): 
        return data.loc[(data.date >= left_boundary) & (data.date < right_boundary)]
    
    def _make_split(self, X):
        
        """Using expanding window"""
        
        split_block = X.date_block_num.max()
        train = X.loc[X.date_block_num < split_block]
        test = X.loc[X.date_block_num == split_block]
        train_x, train_y = train.drop(columns='y').to_numpy(), train['y'].to_numpy()
        test_x, test_y = test.drop(columns='y').to_numpy(), test['y'].to_numpy()
        
        return train_x, train_y, test_x, test_y
    
    def fit(self, model, data, supplementary_tables):
        
        for i in range(len(self.split_dates)-1):
            print(f"Iteration {i}")
            
            # feature engineering
            batch = self._get_batch(data, right_boundary=self.split_dates[i+1])
            fe = FeatureExtractor(batch)
            X = fe.extract(supplementary_tables)   
            train_x, train_y, test_x, test_y = self._make_split(X)
            print("f1")
            # model train
            model.fit(train_x, train_y)
            print("f2")
            y_pred_train = model.predict(train_x)
            y_pred_test = model.predict(test_x)
            print(f"Train shape: {train_x.shape}")
            print(f"Test shape: {test_x.shape}")
            train_metrics = [metric(train_y, y_pred_train) for metric in self.metrics]
            test_metrics = [metric(test_y, y_pred_test) for metric in self.metrics]
            observation  = [f"{model}",
                            f"2013-01-01 to {self.split_dates[i]}",
                            f"{self.split_dates[i]} to {self.split_dates[i+1]}"]
            observation.extend(train_metrics)
            observation.extend(test_metrics)                       
            self.history.loc[len(self.history)] =  observation
            

In [8]:
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape
split_dates = (pd.date_range('2013-02-01', '2015-11-01', freq='M')+timedelta(days=1))[:2]
cv = SaleCV(split_dates, [mse])
en = ElasticNet()
lr = LinearRegression()
cv.fit(lr, data, supplementary_tables)
cv.history

Iteration 0
f1
f2
Train shape: (123158, 23)
Test shape: (63977, 23)


,Model_name,train_window,test_window,train_mean_squared_error,test_mean_squared_error
0,LinearRegression(),2013-01-01 to 2013-03-01 00:00:00,2013-03-01 00:00:00 to 2013-04-01 00:00:00,3.875324,5.489542


In [9]:
from sklearn.svm import SVR
regr = SVR()
cv.fit(regr, data, supplementary_tables)
cv.history

Iteration 0
f1
f2
Train shape: (123158, 23)
Test shape: (63977, 23)


,Model_name,train_window,test_window,train_mean_squared_error,test_mean_squared_error
0,LinearRegression(),2013-01-01 to 2013-03-01 00:00:00,2013-03-01 00:00:00 to 2013-04-01 00:00:00,3.875324,5.489542
1,SVR(),2013-01-01 to 2013-03-01 00:00:00,2013-03-01 00:00:00 to 2013-04-01 00:00:00,10.220886,19.944580


In [ ]:
cv.history